In [198]:
import pandas as pd
import numpy as np

from numpy import asarray
from numpy import savetxt

import re
import csv
import string
import operator
import collections

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

import torch
from transformers import GPT2Tokenizer, pipeline

import gensim

In [186]:
#load dataset and preprocess
twt = pd.read_csv('/Users/irenefontperadejordi/Documents/DOCUMENTS/Beca-La-Caixa/Beca-La-Caixa-Fellowship/13-Cornell-Tech/0_Cornell-Tech-University/Courses/SemesterII/CS5304-Final-Project-Data-Science/train.csv')
twt = twt.set_index('id')

In [187]:
stop_words = stopwords.words('english')
porter = PorterStemmer()

In [190]:
def processing (sentence):
    result = sentence.lower() #Lower case 
    result = re.sub(r'\d+', '', result) #Removing numbers
    result = result.translate(str.maketrans('', '', string.punctuation)) #Remove weird characters
    result = result.strip() #Eliminate blanks from begining and end of setences
    result = result.split() #Separate into words
    result = [w for w in result if not w in stop_words] #Eliminate stop_words
    result = [porter.stem(word) for word in result] #Stem Words
    return (result)

In [193]:
text = list(twt["text"])
target = (list(twt["target"]))

processing(text[0])

['deed', 'reason', 'earthquak', 'may', 'allah', 'forgiv', 'us']

###  Code used to export as a txt

In [155]:
"""
outF = open("hashtag_0.txt", "w")
for line in hashtag_0:
    outF.write(line)
    outF.write("\n")
outF.close()"""

### Dealing with Hashtags

In [194]:
def find_hash (tweet):
    find_hash = re.compile(r"#(\w+)")
    return find_hash.findall(tweet)

In [195]:
hashtag_0 = [] 
hashtag_1 = []

for i in range(len(target)):
    if target[i] == 0:
        hashtag_0.extend(find_hash(text[i]))
    else:
        hashtag_1.extend(find_hash(text[i]))      

In [46]:
hashtag_0_dic = {}
hashtag_1_dic = {} 

for i in hashtag_0:
    if i not in hashtag_0_dic:
        hashtag_0_dic[i] = 0
    hashtag_0_dic[i] += 1

for i in hashtag_1:
    if i not in hashtag_1_dic:
        hashtag_1_dic[i] = 0
    hashtag_1_dic[i] += 1

In [27]:
print(len(hashtag_0))
print(len(hashtag_1))

1688
1642


In [147]:
## check the repeated words
print(len(hashtag_0_dic))
print(len(hashtag_1_dic))

1210
1058


In [178]:
#GET THE HIGHEST FREQ.
sorted_hashtag_0_dic = collections.OrderedDict(sorted(hashtag_0_dic.items(), key=operator.itemgetter(1), reverse=True))
sorted_hashtag_1_dic = collections.OrderedDict(sorted(hashtag_1_dic.items(), key=operator.itemgetter(1), reverse=True))

In [202]:
f = open("sorted_hashtag_0_dic.txt","w")
f.write( str(sorted_hashtag_0_dic) )
f.close()

In [203]:
w = csv.writer(open("sorted_hashtag_0_dic.csv", "w"))
for key, val in sorted_hashtag_0_dic.items():
    w.writerow([key, val])
f.close()

##### Check the Sentiment Analysis for hashtags

In [65]:
sentiment_analysis = pipeline('sentiment-analysis',tokenizer='GPT2Tokenizer')

In [76]:
hashtag_0_sent = []
hashtag_1_sent = []

for i in range(len(hashtag_0)):
    hashtag_0_sent.append(sentiment_analysis(hashtag_0[i])[0])
    
for i in range(len(hashtag_1)):
    hashtag_1_sent.append(sentiment_analysis(hashtag_1[i])[0])

In [130]:
neg_0 = 0
pos_0 = 0

for i in hashtag_0_sent:
    if i["label"] == "POSITIVE":
        pos_0 += 1
    else:
        neg_0 += 1

In [131]:
neg_1 = 0
pos_1 = 0

for i in hashtag_1_sent:
    if i["label"] == "POSITIVE":
        pos_1 += 1
    else:
        neg_1 += 1

In [137]:
print("Tweets classified as 0 have", neg_0, "# NEGATIVES and", pos_0, "# POSITIVES")
print("Tweets classified as 1 have", neg_1, "# NEGATIVES and", pos_1, "# POSITIVES")

Tweets classified as 0 have 637 # NEGATIVES and 1051 # POSITIVES
Tweets classified as 1 have 598 # NEGATIVES and 1044 # POSITIVES


In [140]:
extract_features = pipeline('feature-extraction',tokenizer='GPT2Tokenizer', pad_to_max_length=True)

TypeError: __init__() got an unexpected keyword argument 'pad_to_max_length'

In [139]:
extract_features(hashtag_0, pad_to_max_length=True)

ValueError: The sequences building the batch are not of the same size, no tensor can be built. Set `pad_to_max_length=True` to pad the smaller sequencesup to the larger sequence's length.

### Dealing with @

In [97]:
def find_at (tweet):
    find_at = re.compile(r"@(\w+)")
    return find_at.findall(tweet)

In [100]:
at_0 = [] 
at_1 = []

for i in range(len(target)):
    if target[i] == 0:
        at_0.extend(find_at(text[i]))
    else:
        at_1.extend(find_at(text[i]))      

### Dealing with URL

In [119]:
def find_url (tweet):
    return re.findall(r'(https?://\S+)', tweet)

In [120]:
url_0 = [] 
url_1 = []

for i in range(len(target)):
    if target[i] == 0:
        url_0.extend(find_url(text[i]))
    else:
        url_1.extend(find_url(text[i]))      